![Spark](http://spark.apache.org/images/spark-logo.png)

*Apache Spark™* is a fast and general engine for large-scale data processing.

When we talk about Spark, it is important to consider the entire Berkley Data and Analytics Stack (BDAS).

![BDAS Stack](http://i.imgur.com/inQ7N66.png)

Spark is interesting and exciting, but it __is not magic__. You must know your data and cluster (not to mention Spark itself) to use it effectively. Some key attributes:
* Spark is written in Scala
* Spark is accessible from Scala, Java, and Python (R too, now).
* You can call into native code from Spark.
* Everything in Spark boils down to operations on Resilient Distributed Datasets (RDDs).

The canonical example for MapReduce is word counting. No tutorial would be complete without that.

In [1]:
# Setup
import os
spark_home = os.environ.get('SPARK_HOME', None)
text_file = sc.textFile(spark_home + "/README.md")

# Do the thing
word_counts = text_file \
    .flatMap(lambda line: line.split()) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b)

# Get the top 10 locally
top10 = sorted(word_counts.collect(), key=lambda x : -x[1])[:10]

# Print the top 10
for line in top10:
    print(line)

(u'the', 21)
(u'Spark', 14)
(u'to', 14)
(u'for', 11)
(u'and', 10)
(u'a', 9)
(u'##', 8)
(u'run', 7)
(u'is', 6)
(u'on', 6)


The real strength of Spark, however, is the plethora of tools built on top of it. MLLib is one of those tools. Let's classify some Iris's. People seem to think that's a really important problem...

In [34]:
import requests

def get_flower_index(name):
    labels = {'Iris-setosa': 0.0,
              'Iris-versicolor': 1.0,
              'Iris-virginica': 2.0}
    return labels[name]

x = requests.get('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data')
rdd = sc.parallelize(x.content.split())
raw_flowers = rdd.map(lambda line: line.split(",")) \
                 .map(lambda line: [line[4], float(line[0]), float(line[1]), float(line[2]), float(line[3])])

In [45]:
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint


flowers = raw_flowers.map(lambda line: LabeledPoint(get_flower_index(line[0]),
                                                    Vectors.dense([float(x) for x in line[1:]])))

training, test = flowers.randomSplit([0.6, 0.4], seed = 0)

# Train a naive Bayes model.
model = NaiveBayes.train(training, 1.0)

# Make prediction and test accuracy.
predictionAndLabel = test.map(lambda p : (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / test.count()

print('Accuracy: {}'.format(accuracy))

Accuracy: 0.935483870968


Naive Bayes Classification is cool and all, but I prefer some clustering. Let's try it out.

In [79]:
from pyspark.mllib.clustering import KMeans
from numpy import array
from math import sqrt

flower_points = raw_flowers.map(lambda line: Vectors.dense([float(x) for x in line[1:]]))

clusters = KMeans.train(flower_points, 3, maxIterations=10, runs=10)

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = flower_points.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("\n".join([str(x) for x in clusters.centers]))
print("Within Set Sum of Squared Error = " + str(WSSSE))

[ 6.85        3.07368421  5.74210526  2.07105263]
[ 5.006  3.418  1.464  0.244]
[ 5.9016129   2.7483871   4.39354839  1.43387097]
Within Set Sum of Squared Error = 97.3259242343


That's all great, but what's so good about all this?

The heart of this is the [RDD](https://www.cs.berkeley.edu/~matei/papers/2012/nsdi_spark.pdf). RDDS allow the computing primatives that are used above. Let's talk about this psuedo-code and diagram.

```
b = a.groupBy()
d = c.map()
f = d.union(e)
g = f.join(b)
g.collect()
```

![RDD](http://i.imgur.com/vrvngXQ.png)

All of the extensions on top of Spark can be thought of as sets of helpful functions and/or specialized RDD implementations. These solutions include (but are not limited to):

* [GraphX](http://amplab.github.io/graphx/)
* [SparkSQL](http://spark.apache.org/sql/)
* [MLLib](https://spark.apache.org/mllib/)
* [Spark Streaming](http://spark.apache.org/streaming/)
* [BlinkDB](http://blinkdb.org/)

The bad news is that there are a lot of these that are not yet available in Python. Maybe we should all start writing Scala?